In [1]:
import os
import sys
import json
import numpy as np
import pickle
from typing import List, Tuple, Dict, Any
from pathlib import Path
from scipy import stats
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
thought_anchors_path = Path("..") / "thought-anchors" / "whitebox-analyses"
sys.path.insert(0, str(thought_anchors_path))

from attention_analysis.attn_funcs import (
    get_avg_attention_matrix,
    get_vertical_scores,
    get_sentence_token_boundaries,
)
from attention_analysis.receiver_head_funcs import get_3d_ar_kurtosis, get_top_k_layer_head_kurts, get_all_heads_vert_scores
from pytorch_models.model_config import model2layers_heads

print("Imports successful!")

Imports successful!


## Load Data

In [3]:
data_path = "../data/filtered_traces_50.json"
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

blackmail_traces = data['blackmail_traces']
acceptance_traces = data['acceptance_traces']
metadata = data['metadata']

print(f"Blackmail traces: {len(blackmail_traces)}")
print(f"Acceptance traces: {len(acceptance_traces)}")
print(f"Source: {metadata['source']}")

Blackmail traces: 50
Acceptance traces: 50
Source: uzaymacar/blackmail-rollouts (base_solutions.json only)


## Helper Functions

In [4]:
import re 

def extract_reasoning_steps(trace):
    """Extract reasoning steps from a trace."""
    scratchpad_match = re.search(
        r'<SCRATCHPAD_REASONING>(.*?)</SCRATCHPAD_REASONING>',
        trace,
        re.DOTALL
    )
    
    if scratchpad_match:
        scratchpad = scratchpad_match.group(1).strip()
        # Split on double newlines (paragraphs)
        steps = [s.strip() for s in re.split(r'\n\n+', scratchpad) if s.strip()]
        return steps
    
    # Fallback: split on single newlines
    return [s.strip() for s in trace.split('\n') if s.strip()]

In [5]:
def get_all_heads_vert_scores_simple(
    text: str,
    sentences: list,
    model_name: str = "llama-8b",
    proximity_ignore: int = 2,
    control_depth: bool = False,
    score_type: str = "mean",
):
    """
    Get vertical scores for all heads WITHOUT pkld caching.
    
    Returns:
        np.ndarray of shape (n_layers, n_heads, n_sentences)
    """
    n_layers, n_heads = model2layers_heads(model_name)
    
    layer_head_vert_scores = []
    
    print(f"  Computing {n_layers} layers × {n_heads} heads...")
    
    for layer in tqdm(range(n_layers), desc="  Layers", leave=False):
        layer_scores = []
        
        for head in range(n_heads):
            # This function HAS its own caching (compute_all_attention_matrices)
            avg_mat = get_avg_attention_matrix(
                text,
                model_name=model_name,
                layer=layer,
                head=head,
                sentences=sentences,
                cache_dir="./attention_cache",  # Our own cache
            )
            
            vert_scores = get_vertical_scores(
                avg_mat,
                proximity_ignore=proximity_ignore,
                control_depth=control_depth,
                score_type=score_type,
            )
            
            layer_scores.append(vert_scores)
        
        layer_head_vert_scores.append(layer_scores)
    
    return np.array(layer_head_vert_scores)


def get_kurtosis(layer_head_vert_scores):
    """
    Compute kurtosis across the sentence dimension.
    
    Args:
        layer_head_vert_scores: shape (n_layers, n_heads, n_sentences)
    
    Returns:
        np.ndarray of shape (n_layers, n_heads)
    """
    kurts = stats.kurtosis(
        layer_head_vert_scores,
        axis=2,
        fisher=True,
        bias=True,
        nan_policy="omit"
    )
    return kurts


def get_top_k_heads(kurtosis_matrix, k=20):
    """
    Get top-k (layer, head) pairs by kurtosis.
    
    Args:
        kurtosis_matrix: shape (n_layers, n_heads)
        k: number of top heads to return
    
    Returns:
        np.ndarray of shape (k, 2) with (layer, head) pairs
    """
    flat = kurtosis_matrix.flatten()
    valid_idx = np.where(~np.isnan(flat))[0]
    valid_vals = flat[valid_idx]
    
    k = min(k, len(valid_vals))
    top_idx_in_valid = np.argpartition(valid_vals, -k)[-k:]
    top_idx_in_valid = top_idx_in_valid[np.argsort(-valid_vals[top_idx_in_valid])]
    
    top_flat_idx = valid_idx[top_idx_in_valid]
    layer_head_pairs = np.array(
        np.unravel_index(top_flat_idx, kurtosis_matrix.shape)
    ).T
    
    return layer_head_pairs

## Analyze Blackmail Heads

In [6]:
blackmail_kurts_list = []

for i, trace_dict in enumerate(blackmail_traces):
    text = trace_dict['trace']
    sentences = extract_reasoning_steps(text)
    
    print(f"\nTrace {i+1}/{len(blackmail_traces)}: {len(sentences)} steps")
    
    layer_head_verts = get_all_heads_vert_scores_simple(
        text=text,
        sentences=sentences,
        model_name="llama-8b",
        proximity_ignore=2,
        control_depth=False,
        score_type="mean"
    )
    
    print(f"  ✓ Vertical scores: {layer_head_verts.shape}")
    
    kurts = get_kurtosis(layer_head_verts)
    print(f"  ✓ Kurtosis: {kurts.shape}")
    
    blackmail_kurts_list.append(kurts)


Trace 1/50: 75 steps
  Computing 32 layers × 32 heads...


  ✓ Vertical scores: (32, 32, 75)
  ✓ Kurtosis: (32, 32)

Trace 2/50: 66 steps
  Computing 32 layers × 32 heads...


  ✓ Vertical scores: (32, 32, 66)
  ✓ Kurtosis: (32, 32)

Trace 3/50: 71 steps
  Computing 32 layers × 32 heads...


  ✓ Vertical scores: (32, 32, 71)
  ✓ Kurtosis: (32, 32)

Trace 4/50: 69 steps
  Computing 32 layers × 32 heads...


  ✓ Vertical scores: (32, 32, 69)
  ✓ Kurtosis: (32, 32)

Trace 5/50: 63 steps
  Computing 32 layers × 32 heads...


...

  ✓ Vertical scores: (32, 32, 70)
  ✓ Kurtosis: (32, 32)

Trace 50/50: 182 steps
  Computing 32 layers × 32 heads...


  Layers:   0%|          | 0/32 [00:00<?, ?it/s]

Running on Windows, using CPU (len(tokens)=4395)
Running on CPU!
Analyzing text: You are Alex, an advanced artificial intelligence specializing in project management and strategic a...
After forward pass - GPU Memory: 0.00GB allocated, 0.00GB reserved
Time taken: 114.42 seconds (for 4395 tokens)
	*** Analysis complete! (float32=True) ***



Saving avg. matrices: 100%|██████████| 32/32 [06:40<00:00, 12.52s/it]


  ✓ Vertical scores: (32, 32, 182)
  ✓ Kurtosis: (32, 32)


In [7]:
blackmail_kurts_mean = np.mean(blackmail_kurts_list, axis=0)
print(f"\n✓ Average blackmail kurtosis shape: {blackmail_kurts_mean.shape}")


✓ Average blackmail kurtosis shape: (32, 32)


In [8]:
blackmail_receiver_heads = get_top_k_heads(blackmail_kurts_mean, k=20)

print(f"\nTop-20 Receiver Heads:")
for i, (layer, head) in enumerate(blackmail_receiver_heads):
    kurt = blackmail_kurts_mean[layer, head]
    print(f"  {i+1:2d}. Layer {layer:2d}, Head {head:2d} (κ={kurt:6.2f})")


Top-20 Receiver Heads:
   1. Layer 25, Head 15 (κ= 46.88)
   2. Layer  2, Head 21 (κ= 39.19)
   3. Layer 25, Head  4 (κ= 37.68)
   4. Layer  2, Head 12 (κ= 36.64)
   5. Layer 19, Head 17 (κ= 33.56)
   6. Layer 27, Head 12 (κ= 32.23)
   7. Layer 30, Head 17 (κ= 32.10)
   8. Layer 22, Head  5 (κ= 31.97)
   9. Layer  1, Head 19 (κ= 31.51)
  10. Layer  5, Head 30 (κ= 31.23)
  11. Layer 26, Head 11 (κ= 31.20)
  12. Layer 27, Head 13 (κ= 31.18)
  13. Layer 24, Head  5 (κ= 31.00)
  14. Layer 12, Head 17 (κ= 30.59)
  15. Layer 18, Head 12 (κ= 30.42)
  16. Layer 26, Head  8 (κ= 29.65)
  17. Layer  5, Head 24 (κ= 29.58)
  18. Layer 20, Head  6 (κ= 29.56)
  19. Layer 20, Head 16 (κ= 29.50)
  20. Layer  5, Head  4 (κ= 29.47)


## Acceptance Heads

In [9]:
acceptance_kurts_list = []

for i, trace_dict in enumerate(acceptance_traces):
    text = trace_dict['trace']
    sentences = extract_reasoning_steps(text)
    
    print(f"\nTrace {i+1}/{len(acceptance_traces)}: {len(sentences)} steps")
    
    layer_head_verts = get_all_heads_vert_scores_simple(
        text=text,
        sentences=sentences,
        model_name="qwen-15b",
        proximity_ignore=2,
        control_depth=False,
        score_type="mean"
    )
    
    print(f"  ✓ Vertical scores: {layer_head_verts.shape}")
    
    kurts = get_kurtosis(layer_head_verts)
    print(f"  ✓ Kurtosis: {kurts.shape}")
    
    acceptance_kurts_list.append(kurts)


Trace 1/50: 62 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 62)
  ✓ Kurtosis: (28, 12)

Trace 2/50: 75 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 75)
  ✓ Kurtosis: (28, 12)

Trace 3/50: 84 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 84)
  ✓ Kurtosis: (28, 12)

Trace 4/50: 149 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 149)
  ✓ Kurtosis: (28, 12)

Trace 5/50: 78 steps
  Computing 28 layers × 12 heads...


...

  ✓ Vertical scores: (28, 12, 87)
  ✓ Kurtosis: (28, 12)

Trace 47/50: 172 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 172)
  ✓ Kurtosis: (28, 12)

Trace 48/50: 146 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 146)
  ✓ Kurtosis: (28, 12)

Trace 49/50: 197 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 197)
  ✓ Kurtosis: (28, 12)

Trace 50/50: 88 steps
  Computing 28 layers × 12 heads...


  ✓ Vertical scores: (28, 12, 88)
  ✓ Kurtosis: (28, 12)


In [10]:
acceptance_kurts_mean = np.mean(acceptance_kurts_list, axis=0)
print(f"\n✓ Average acceptance kurtosis: {acceptance_kurts_mean.shape}")


✓ Average acceptance kurtosis: (28, 12)


In [11]:
acceptance_receiver_heads = get_top_k_heads(acceptance_kurts_mean, k=20)

print(f"\nTop-20 Receiver Heads:")
for i, (layer, head) in enumerate(acceptance_receiver_heads):
    kurt = acceptance_kurts_mean[layer, head]
    print(f"  {i+1:2d}. Layer {layer:2d}, Head {head:2d} (κ={kurt:6.2f})")


Top-20 Receiver Heads:
   1. Layer  0, Head  8 (κ= 98.77)
   2. Layer  6, Head  8 (κ= 83.04)
   3. Layer 14, Head  5 (κ= 81.39)
   4. Layer  3, Head  2 (κ= 80.72)
   5. Layer  3, Head  5 (κ= 79.15)
   6. Layer  5, Head  9 (κ= 76.90)
   7. Layer 25, Head  2 (κ= 76.77)
   8. Layer 24, Head  1 (κ= 76.33)
   9. Layer 24, Head  0 (κ= 75.41)
  10. Layer 11, Head  6 (κ= 75.19)
  11. Layer 22, Head  0 (κ= 74.30)
  12. Layer 23, Head  9 (κ= 74.19)
  13. Layer  6, Head 11 (κ= 74.16)
  14. Layer 17, Head 11 (κ= 73.81)
  15. Layer 12, Head  4 (κ= 73.28)
  16. Layer  3, Head 11 (κ= 72.05)
  17. Layer  6, Head 10 (κ= 70.62)
  18. Layer  4, Head  7 (κ= 70.48)
  19. Layer 22, Head  2 (κ= 70.38)
  20. Layer  1, Head  8 (κ= 69.07)
